In [1]:
from googleapiclient.discovery import build
import pandas as pd
import time
import os
from IPython.display import JSON

In [10]:
def get_video_details(video_id, youtube):    

    #collect view, like, dislike, favourite, comment counts
    request = youtube.videos().list(
        part="id,statistics",
        id= video_id
    )
    response_video_stats = request.execute()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount'] if 'dislikeCount' in response_video_stats['items'][0]['statistics'] else 0
    favorite_count = response_video_stats['items'][0]['statistics']['favoriteCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, dislike_count, favorite_count, comment_count

In [11]:
def get_video_stats(df, CHANNEL_ID):
    #make API call
    pageToken = ""
            
    api_service_name = "youtube"
    api_version = "v3"
    
    # Get credentials and create an API client
    youtube = build(
        api_service_name, api_version, developerKey=API_KEY)


    request = youtube.search().list(
            part="snippet, id",
            channelId=CHANNEL_ID,
            maxResults=10000,
            order="date",            
        )
    response = request.execute()
    time.sleep(3)

    for video in response['items']:
        
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;", "")
                upload_time = video['snippet']['publishedAt']
                upload_date = str(upload_time).split("T")[0]
            
                view_count, like_count, dislike_count,favorite_count, comment_count = get_video_details(video_id, youtube)

                #save data in pandas df
                df = df.append({'video_id': video_id, 'video_title':video_title,
                                'upload_date':upload_date, 'view_count':view_count,
                                'like_count':like_count, 'dislike_count':dislike_count,
                                'favorite_count':favorite_count, 'comment_count':comment_count}, ignore_index=True)

    pageToken = response.get('nextPageToken')

    while pageToken is not None:
    
        request = youtube.search().list(
            part="snippet, id",
            channelId=CHANNEL_ID,
            maxResults=10000,
            order="date",
            pageToken=pageToken,
        )
        response = request.execute()
        time.sleep(3)

        for video in response['items']:
        
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;", "")
                upload_time = video['snippet']['publishedAt']
                upload_date = str(upload_time).split("T")[0]
            
                view_count, like_count, dislike_count,favorite_count, comment_count = get_video_details(video_id, youtube)

                #save data in pandas df
                df = df.append({'video_id': video_id, 'video_title':video_title,
                                'upload_date':upload_date, 'view_count':view_count,
                                'like_count':like_count, 'dislike_count':dislike_count,
                                'favorite_count':favorite_count, 'comment_count':comment_count}, ignore_index=True)

        pageToken = response.get('nextPageToken')
    return df

In [ ]:
#main
#Keys
API_KEY = os.environ.get("YOUTUBE_APIKEY")
CHANNEL_ID = "UCW8Ews7tdKKkBT6GdtQaXvQ" 

#build dataframe
df = pd.DataFrame(columns= ['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'dislike_count', 'favorite_count','comment_count'])

df = get_video_stats(df, CHANNEL_ID)

df.to_csv('video_stats2.csv')

In [ ]:
df.head()